In [ ]:
import argparse
import csv
import itertools
import os
import os.path

import joblib
from cyvcf2 import VCF
import gzip
import re

In [ ]:
path_input = "./data/raw/gnomAD/v2/hg19/gnomad.exomes.r2.1.1.sites.1.vcf.bgz"
#path_input = "./data/raw/jMorp/hg19/tommo-8.3kjpn-20200831-af_snvall-autosome.vcf.gz"
seq = "hg19"
path_ref = "./data/processed/VCF/Columns_gnomAD.txt"
#path_ref = "./data/processed/VCF/Columns_jMorp.txt"
path_output = "./data/processed/VCF"

In [ ]:
# --- Load data file
# Input data
#path_input = args.input
basename = os.path.basename(path_input)
print("Input vcf file name : {}".format(os.path.basename(path_input)))
with gzip.open(path_input, 'rb') as f:
    line = f.readline()
    line = line.decode().strip()
    vcf_version = line.lstrip("##fileformat=")
    print("--> VCF version: {}".format(vcf_version))

In [ ]:
# Additional data
#path_ref = args.ref
with open(path_ref, newline="") as f:
    reader = csv.reader(f, delimiter="\t")
    column_list = list(reader)
vcf_additional = list(itertools.chain.from_iterable(column_list))
print("Additional vcf column list file name : {}".format(os.path.basename(path_ref)))

In [ ]:
def format_str(x):
    if x is None:
        y = ''
    else:
        if isinstance(x, list):
            if not x:
                y = ';'.join(x)
            else:
                y = ''
        else:
            y = str(x)
    return y

In [ ]:
def info(record, target, i, out):
    x = record.INFO.get(target)
    if isinstance(x, list):
        check_num = len(x) - 1
        if check_num == i:
            r = x[i]
            out.append(r)
        else:
            i = check_num - 1
            r = x[i]
            out.append(r)
    else:
        out.append(x)
    return out

In [ ]:
def picker(variant, vcf_columns):
    df = []
    # vcf_template
    chrom = variant.CHROM
    pos = variant.POS
    variant_id = variant.ID
    ref = variant.REF
    alt = variant.ALT
    num = len(alt)
    for i in range(num):
        a = alt[i]
        new_record = [chrom, pos, variant_id, ref, a]
        new_record = list(map(format_str, new_record))
        # vcf_additional
        col = vcf_columns[5:]
        new_record = [info(variant, str(c), i, new_record) for c in col]
        df.append(new_record)
    df = [flatten for inner in df for flatten in inner]
    return df

In [ ]:
def parse_vcf(vcf_file, vcf_columns, vcf_version):
    # For jMorp (allows UTF-8) or gnomAD (ASCII only)
    if vcf_version == "VCFv4.3" or vcf_version == "VCFv4.2":
        df = [picker(variant, vcf_columns) for variant in VCF(vcf_file)]
        df = [flatten for inner in df for flatten in inner]
        df = list(map(list, set(map(tuple, df))))
    else:
        raise Exception("[Error] {} is not verified!".format(version))
    return df

---

In [ ]:
# --- Parse VCF file
print("Parsing vcf file...")
vcf_template = ['CHROM', 'POS', 'ID', 'REF', 'ALT']
vcf_columns = vcf_template + vcf_additional
vcf = parse_vcf(path_input, vcf_columns, vcf_version)

In [ ]:
# --- Convert to DataFrame
import pandas as pd
vcf = pd.DataFrame(vcf)
vcf.columns = vcf_columns

In [ ]:
vcf.head(10)

In [ ]:
# --- Export file
# Pickle file
print("Pickling parsed vcf file...")
pickled_path = "{0}/{1}/{2}.pkl".format(path_output, seq, os.path.splitext(basename)[0])
joblib.dump(vcf, pickled_path)

---

In [ ]:
import pandas as pd
#pickled_input = "./data/processed/VCF/hg19/gnomad.exomes.r2.1.1.sites.1.vcf.pkl"
vcf_data = joblib.load(pickled_path)
vcf_data = pd.DataFrame(vcf_data)
vcf_columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT',
               'AF', 'AF_MALE', 'AF_FEMALE',
               'AC', 'AC_MALE', 'AC_FEMALE',
               'AN', 'AN_MALE', 'AN_FEMALE',
               'AF_afr', 'AF_amr', 'AF_asj',
               'AF_eas', 'AF_fin', 'AF_nfe',
               'AF_oth', 'AC_eas', 'AN_eas']
vcf_data.columns = vcf_columns
vcf_data.head()